In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

**Extract Datset**

In [2]:
!mkdir lfwDataSet
!unzip ./gdrive/My\ Drive/lfw-dataset.zip -d lfwDataSet
!unzip ./lfwDataSet/lfw-deepfunneled.zip -d lfwDataSet

Output hidden; open in https://colab.research.google.com to view.

In [3]:
import cv2
from imutils import paths
import random
import keras.layers as KL
import pandas as pd
from keras.preprocessing.image import img_to_array
import sklearn.model_selection
import numpy as np
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
import imutils
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.legacy.layers import MaxoutDense
from keras import backend as K
from skimage import io
import os
from keras.models import Model
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Input
from keras import regularizers
import tensorflow as tf

Using TensorFlow backend.


# Data Preparation


In [0]:
def getPeopleWithSomeThresh(people,lowThresh=8,highThresh=100):
  mask=people['images']>lowThresh
  filPeople=people[mask]
  mask=filPeople['images']<highThresh
  return filPeople[mask]

def loadImages(names,data_dir='./lfwDataSet/lfw-deepfunneled'):
  loadedImages = []
#   lp = []
  for n in names:
    path=os.path.join(data_dir,n)
#     imagesList = listdir(path)
#     for image in imagesList:
    img_paths=[os.path.join(path,img) for img in os.listdir(path)]
    person_images=io.imread_collection(img_paths)
#     person_images=io.imread_collection(img_paths)
#     img = io.imread(os.path.join(path,image))
    loadedImages.append(person_images)
#       lp.append(n)
   
  return np.array(loadedImages)#,np.array(lp)

def getTrainingData(people,possThresh,highnegThresh):
  possPeople = getPeopleWithSomeThresh(people,possThresh)
  possPeopleImages = loadImages(possPeople['name'])
  negPeople = getPeopleWithSomeThresh(people,1,highnegThresh)
  negPeopleImages = loadImages(negPeople['name'])
  return possPeople,possPeopleImages,negPeople,negPeopleImages
  

**Data Loading**

In [0]:
data_dir="./lfwDataSet"
possThresh = 5
highnegThresh = 3
people=pd.read_csv(os.path.join(data_dir,"lfw_allnames.csv"))
possPeople,possPeopleImages,negPeople,negPeopleImages = getTrainingData(people,possThresh,highnegThresh)


In [6]:
print (possPeopleImages.shape,negPeopleImages.shape)

(306,) (779, 2, 250, 250, 3)


In [7]:
print (np.array(possPeopleImages[5]).shape)

(39, 250, 250, 3)


In [0]:
#creating training data
def getMiniBatch(possImages,numPeoplePerBatch=15,numImagesPerPerson=8,negImages=None,numNegImages=40):
    
    ## PossPeople is a dataframe containg name and number of images of a person
    ## negPeople is a  //   //   //  //  //  //  //   //   //  //    //   //  //
    ## MiniBatch = numPeoplePerBatch*numImages
    ## numNegImages   Total number of negative images in the end of a batch
    
    num_people=len(possImages)
    num_batches_per_epoch=num_people//numPeoplePerBatch
    batch_num=0   
    person_indices=np.arange(num_people)
    while True:
        #Iteration over all Training Data
        if batch_num%num_batches_per_epoch==0:
            np.random.shuffle(person_indices)
            
            batch_num=0
        personStartIndex=batch_num*numPeoplePerBatch    ## Starting Index in the start of each batch
        batchImages=[]
        for i in np.arange(numPeoplePerBatch):
            pickAnyPersonIndex=person_indices[personStartIndex+i]
            num_images_for_person=len(possImages[pickAnyPersonIndex])   ## Picking Random Person Images
            imgs_to_pick=np.random.randint(0,num_images_for_person,numImagesPerPerson)   ## Pick images equal to numImagesPerPerson
            person_imgs=np.stack(possImages[pickAnyPersonIndex],axis=0)
            batchImages.append(person_imgs[imgs_to_pick])
            
        batch_neg_imgs=[]
        total_neg_people=len(negImages)
        neg_person_inds=np.random.randint(0,total_neg_people,numNegImages)
        for i in neg_person_inds:
            neg_person_index_orig=i
            num_images_for_neg_person=len(negImages[neg_person_index_orig])
            neg_imgs_to_pick=np.random.randint(0,num_images_for_neg_person,1)
            neg_person_imgs=np.stack(negImages[neg_person_index_orig],axis=0)
            batch_neg_imgs.append(neg_person_imgs[neg_imgs_to_pick])
        h,w=250,250
        batchImages=np.stack(batchImages,axis=0).reshape(-1,h,w,3)
        batch_neg_imgs=np.stack(batch_neg_imgs,axis=0).reshape(-1,h,w,3)
        batchImages=np.concatenate((batchImages,batch_neg_imgs))
        batch_num+=1
        
        
        yield batchImages,[]
#         print(batchImages.shape)
#         return batchImages,[]


In [0]:
num_people_per_batch=10
num_images_per_person=possThresh
num_negs=50
num_batces_per_epoch=len(possPeople)//num_people_per_batch
num_epochs=500
embedding_size=128
margin=5.0
batch=getMiniBatch(possPeopleImages,num_people_per_batch,
                        num_images_per_person,negPeopleImages,num_negs)


In [0]:
#L2 Distance
def l2(x,y):
    return K.sum(K.square(x-y), axis=-1)


def tripletLossOnline(batch_embeddings,num_persons,num_images_per_person,num_negs,embedding_size,margin):
    num_persons=tf.cast(num_persons,tf.int32)
    num_images_per_person=persons=tf.cast(num_images_per_person,tf.int32)
    num_negs=tf.cast(num_negs,tf.int32)
    embedding_size=tf.cast(embedding_size,tf.int32)
   

    num_anchors=num_persons*num_images_per_person    #Split Positive and Negatives
    anchors,negs=tf.split(batch_embeddings,[num_anchors,num_negs])
   
    
    #Calculating L2 distance between anchor and positives
    anchors1=tf.reshape(anchors,(num_persons,num_images_per_person,1,embedding_size)) 
    anchors2=tf.reshape(anchors,(num_persons,1,num_images_per_person,embedding_size))
    l2_all_pos=l2(anchors1,anchors2)
    
    
    #Negative Distances
    l2_all_negs=l2(tf.reshape(anchors,(-1,1,embedding_size)),negs)
    
    
    reshaped_l2_all_negs=tf.reshape(l2_all_negs,(num_persons,num_images_per_person,1,num_negs)) #Resahping -ve distances
    tiled_l2_all_negs=tf.tile(reshaped_l2_all_negs,(1,1,num_images_per_person,1))  #Tiling -ve distances
    
    #reshaping l2 all pos to num_persons*num_images_per_person*num_images_per_person,1
    l2_all_pos=tf.reshape(l2_all_pos,(-1,1))
    l2_all_negs=tf.reshape(tiled_l2_all_negs,(-1,num_negs))    
    
    #Calculating Loss
    loss=l2_all_pos-l2_all_negs
    inds=tf.where(loss<0) #loss zero where loss is negative
    loss=tf.gather_nd(loss,inds) 
    loss=loss+margin
    loss=tf.maximum(loss,0.0)
    
    loss = K.switch(tf.size(loss) > 0, K.mean(loss), tf.constant(0.0))
    return loss
    

# Facenet Model

In [11]:
num_people_per_batch_t=K.constant(num_people_per_batch,dtype=tf.int32)
num_images_per_person_t=K.constant(num_images_per_person,dtype=tf.int32)
num_negs_t=K.constant(num_negs,dtype=tf.int32)
embedding_size_t=K.constant(embedding_size,dtype=tf.int32)
margin_t=K.constant(margin,dtype=tf.float32)




# model=Sequential()
input_shape = Input(shape = (250, 250, 3))
conv1=KL.Conv2D(64, (7, 7), strides=(2,2),padding="same",activation='relu', input_shape=(250, 250, 3),name="conv1")(input_shape)
# model.add(Conv2D(64, (7, 7), strides=(2,2),padding="same",activation='relu', input_shape=(250, 250, 3),name="conv1"))
# # model.add(BatchNormalization())
# # model.add(Activation("relu"))
mpool = KL.MaxPooling2D(name="pool1")(conv1)
# model.add(MaxPooling2D(name="pool1"))
# # model.add(normalization.LRN2D())
bn = KL.BatchNormalization(name="rnorm1")(mpool)
# model.add(BatchNormalization(name="rnorm1"))
conv2a = KL.Conv2D(64, (1, 1), strides=(1,1),padding="same",activation='relu',name="conv2a")(bn)
# model.add(Conv2D(64, (1, 1), strides=(1,1),padding="same",activation='relu',name="conv2a"))
conv2 = KL.Conv2D(192, (3, 3), strides=(1,1),padding="same",activation='relu',name="conv2")(conv2a)
# model.add(Conv2D(192, (3, 3), strides=(1,1),padding="same",activation='relu',name="conv2"))
rnorm2 = KL.BatchNormalization(name="rnorm2")(conv2)
# model.add(BatchNormalization(name="rnorm2"))
pool2 = KL.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same",name="pool2")(rnorm2)
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same",name="pool2"))
conv3a = KL.Conv2D(192, (1, 1), strides=(1,1),padding="same",activation='relu',name="conv3a")(pool2)
# model.add(Conv2D(192, (1, 1), strides=(1,1),padding="same",activation='relu',name="conv3a"))
conv3 = KL.Conv2D(384, (3, 3), strides=(1,1),padding="same",activation='relu',name="conv3")(conv3a)
# model.add(Conv2D(384, (3, 3), strides=(1,1),padding="same",activation='relu',name="conv3"))
pool3 = KL.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same",name="pool3")(conv3)
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same",name="pool3"))
conv4a = KL.Conv2D(384, (1, 1), strides=(1,1),padding="same",activation='relu',name="conv4a")(pool3)
# model.add(Conv2D(384, (1, 1), strides=(1,1),padding="same",activation='relu',name="conv4a"))
conv4 = KL.Conv2D(256, (3, 3), strides=(1,1),padding="same",activation='relu',name="conv4")(conv4a)
# model.add(Conv2D(256, (3, 3), strides=(1,1),padding="same",activation='relu',name="conv4"))
conv5a = KL.Conv2D(256, (1, 1), strides=(1,1),padding="same",activation='relu',name="conv5a")(conv4)
# model.add(Conv2D(256, (1, 1), strides=(1,1),padding="same",activation='relu',name="conv5a"))
conv5 = KL.Conv2D(256, (3, 3), strides=(1,1),padding="same",activation='relu',name="conv5")(conv5a)
# model.add(Conv2D(256, (3, 3), strides=(1,1),padding="same",activation='relu',name="conv5"))
conv6a = KL.Conv2D(256, (1, 1), strides=(1,1),padding="same",activation='relu',name="conv6a")(conv5)
# model.add(Conv2D(256, (1, 1), strides=(1,1),padding="same",activation='relu',name="conv6a"))
conv6 = KL.Conv2D(256, (3, 3), strides=(1,1),padding="same",activation='relu',name="conv6")(conv6a)
# model.add(Conv2D(256, (3, 3), strides=(1,1),padding="same",activation='relu',name="conv6"))
pool4 = KL.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same",name="pool4")(conv6)
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same",name="pool4"))
f = KL.Flatten()(pool4)
# model.add(Flatten())
d1 = KL.MaxoutDense(32*128, 2)(f)
# model.add(MaxoutDense(32*128, 2))
d2 = KL.MaxoutDense(32*128,2)(d1)
# model.add(MaxoutDense(32*128,2))
embeddings = KL.Dense(128,activation='relu')(d2)
# embeddings = KL.Dense(128, activity_regularizer=regularizers.l2(0.05))(d2)
# embeddings = model.add(Dense(128, activity_regularizer=regularizers.l2(0.01)))
loss=KL.Lambda(lambda x: tripletLossOnline(*x), name="l2_loss")(
                [embeddings,num_people_per_batch_t,num_images_per_person_t,num_negs_t,embedding_size_t,margin_t])   
# loss=KL.Lambda(lambda x: triplet_loss_graph(*x), name="l2_loss")(
#                 [batch_all_triplet_loss(labels, embeddings, 0.5, squared=False)])
# model.add(Conv2D(20, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Flatten())
# model.add(Dense(30,activation='relu'))
# model.add(Dense(2,activation='softmax'))
# augmentor = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
# 	height_shift_range=0.1, shear_range=0.2, zoom_range=0.4,
# 	horizontal_flip=True, fill_mode="nearest")

model = Model(input=input_shape, output=embeddings)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 250, 250, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 125, 125, 64)      9472      
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 62, 62, 64)        0         
_________________________________________________________________
rnorm1 (BatchNormalization)  (None, 62, 62, 64)        256       
_________________________________________________________________
conv2a (Conv2D)              (None, 62, 62, 64)        4160      
_________________________________________________________________
conv2 (Conv2D)               (None, 62, 62, 192)       110784    
_________________________________________________________________
rnorm2 (BatchNormalization)  (None, 62, 62, 192)       768       
__________

/usr/local/lib/python3.6/dist-packages/keras/legacy/layers.py:74: UserWarning: The `MaxoutDense` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `MaxoutDense` layer is deprecated '
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
model.add_loss(loss)
sgd = keras.optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
# print (model.summary())
# model.compile(loss=None,optimizer=keras.optimizers.SGD(lr=0.0001),)
model.compile(loss=None,optimizer=sgd)
model.metrics_names.append("l2 loss")
model.metrics_tensors.append(loss)

In [13]:
model.fit_generator(batch,steps_per_epoch=num_batces_per_epoch, epochs=num_epochs)

Epoch 1/500
30/30 [==============================] - 34s 1s/step - loss: 2.4931 - l2 loss: 2.4931
Epoch 2/500
30/30 [==============================] - 23s 783ms/step - loss: 0.0808 - l2 loss: 0.0808
Epoch 3/500
30/30 [==============================] - 23s 780ms/step - loss: 0.0363 - l2 loss: 0.0363
Epoch 4/500
30/30 [==============================] - 23s 778ms/step - loss: 0.0297 - l2 loss: 0.0297
Epoch 5/500
30/30 [==============================] - 23s 779ms/step - loss: 0.0274 - l2 loss: 0.0274
Epoch 6/500
30/30 [==============================] - 23s 780ms/step - loss: 0.0241 - l2 loss: 0.0241
Epoch 7/500
30/30 [==============================] - 23s 779ms/step - loss: 0.0233 - l2 loss: 0.0233
Epoch 8/500
30/30 [==============================] - 23s 780ms/step - loss: 0.0217 - l2 loss: 0.0217
Epoch 9/500
30/30 [==============================] - 23s 781ms/step - loss: 0.0229 - l2 loss: 0.0229
Epoch 10/500
30/30 [==============================] - 23s 782ms/step - loss: 0.0231 - l2 loss:

KeyboardInterrupt: ignored

In [0]:
model.save_weights("weights.h5")

In [15]:
# print(possPeopleImages[0])
for i in possPeopleImages:
  print(type(i[0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

In [16]:
oldImages = []
for i in possPeopleImages:
  oldImages.append(i[0])
oldImages = np.array(oldImages)
print (oldImages.shape)

(306, 250, 250, 3)


In [17]:
newImages = []
for i in possPeopleImages:
  newImages.append(i[1])
newImages = np.array(newImages)
print (newImages.shape)

(306, 250, 250, 3)


In [18]:
oldEmbeddings = model.predict(oldImages) ## [0:1]
newEmbeddings = model.predict(newImages) ## [38:39]
print (oldEmbeddings)
print (newEmbeddings)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.44843483 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.29039723 0.         0.         ... 0.         0.         0.        ]]
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.33802962 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [19]:
print(np.sqrt(np.sum((oldEmbeddings-newEmbeddings)**2)))

98.8912


In [0]:
from skimage import data, io
from matplotlib import pyplot as plt
# for oi in oldImages:
io.imshow(oldImages[0])
plt.show()
io.imshow(newImages[52])
plt.show()

/usr/local/lib/python3.6/dist-packages/skimage/io/_plugins/matplotlib_plugin.py:51: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  out_of_range_float = (np.issubdtype(image.dtype, np.float) and


In [0]:
# dists = np.zeros((newEmbeddings.shape[0], oldEmbeddings.shape[0]))
# for i in range(newEmbeddings.shape[0]):
# #   print(newEmbeddings[0]-oldEmbeddings)aa
#   dists[i,:] = np.sqrt(np.sum((newEmbeddings[i,:] - oldEmbeddings)**2,axis = 1))
# # print (dists)
# # indices = np.argmin(dists,axis=1)
# print (dists)
# dists = l2(oldEmbeddings,newEmbeddings)
# print (np.sum(np.array(possPeople['name'].iloc[indices])==np.array(possPeople['name'])))


indices = []
for i in range(oldEmbeddings.shape[0]):
  dists = []
  for j in range(newEmbeddings.shape[0]):
    dists.append(np.sqrt(np.sum((oldEmbeddings[i]-newEmbeddings[j])**2)))
  dists = np.array(dists)
  indices.append(np.argmin(dists))
    

In [0]:
print (indices)

In [0]:
print (dists.shape)
print (indices)
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [0]:
!tar -xf /content/gdrive/My\ Drive/dataset.tar.xz

In [0]:
model1=Sequential()
# input_shape = Input(shape = (250, 250, 3))
# conv1=KL.Conv2D(64, (7, 7),activation='relu', strides=2,padding="same",name="conv1")(input_shape)
model1.add(Conv2D(64
                  , (7, 7), strides=(2,2),padding="same",activation='relu', input_shape=(250, 250, 3),name="conv1"))
model1.summary()

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense ,Flatten
from keras.layers.convolutional import *
model_same=Sequential([
    Dense(16,activation="relu",input_shape=(250,250,3)),
    Conv2D(32,kernel_size=(7,7),strides=(2,2),activation="relu",padding="same"),
])
print(model_same.summary())